In [5]:

!pip install paddlepaddle-gpu
!pip install paddleocr
!pip install opencv-python-headless

In [6]:
import cv2
import sys
import re
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
from paddleocr import PaddleOCR
import torch
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import re
# import constants
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

In [7]:
!mkdir images

mkdir: cannot create directory ‘images’: File exists


In [8]:
df = pd.read_csv('/content/train.csv')

In [9]:
df.shape

(263859, 4)

# RANGE

In [10]:
df = df[0:10]

In [11]:

def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)


In [12]:
download_images(df['image_link'], '/content/images')

100%|██████████| 10/10 [00:00<00:00, 56.13it/s]


In [13]:
def get_file_name(url):
    return re.search(r'/([^/]+\.[a-z]+)$', url).group(1)

In [14]:
df['filename'] = df.image_link.apply(get_file_name)

In [15]:
# Check if GPU is available and print status
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# PaddleOCR uses GPU if specified
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)


# Folder path
base_image_path = Path('/content/images')


# Function to extract text using PaddleOCR (GPU supported)
def extract_text_paddleocr(filename):
    image_path = base_image_path / filename
    try:
        result = paddle_ocr.ocr(str(image_path))
        text = ' '.join([line[1][0] for line in result[0]])
        return text
    except Exception as e:
        print(f"PaddleOCR failed for {filename}: {e}")
        return None

df['paddleocr'] = df['filename'].apply(extract_text_paddleocr)

Using device: cpu
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:16<00:00, 237kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:18<00:00, 551kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 142kiB/s]

[2024/09/15 05:21:12] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

[2024/09/15 05:21:15] ppocr DEBUG: dt_boxes num : 18, elapsed : 1.1682860851287842
[2024/09/15 05:21:15] ppocr DEBUG: cls num  : 18, elapsed : 0.08907341957092285
[2024/09/15 05:21:19] ppocr DEBUG: rec_res num  : 18, elapsed : 3.1754229068756104
[2024/09/15 05:21:19] ppocr DEBUG: dt_boxes num : 17, elapsed : 0.4699430465698242
[2024/09/15 05:21:19] ppocr DEBUG: cls num  : 17, elapsed : 0.10836410522460938
[2024/09/15 05:21:21] ppocr DEBUG: rec_res num  : 17, elapsed : 1.4925439357757568
[2024/09/15 05:21:21] ppocr DEBUG: dt_boxes num : 53, elapsed : 0.3422536849975586
[2024/09/15 05:21:21] ppocr DEBUG: cls num  : 53, elapsed : 0.1588730812072754
[2024/09/15 05:21:28] ppocr DEBUG: rec_res num  : 53, elapsed : 6.485193490982056
[2024/09/15 05:21:28] ppocr DEBUG: dt_boxes num : 44, elapsed : 0.3248779773712158
[2024/09/15 05:21:28] ppocr DEBUG: cls num  : 44, elapsed : 0.16465973854064941
[2024/09/15 05:21:35] ppocr DEBUG: rec_res num  : 44, elapsed : 6.627528429031372
[2024/09/15 05:21:3

In [16]:
df.to_csv('df1.csv', index=False)

In [17]:
df.sample(5)

,image_link,group_id,entity_name,entity_value,filename,paddleocr
7,https://m.media-amazon.com/images/I/71DiLRHeZd...,731432,item_weight,1400 milligram,71DiLRHeZdL.jpg,VEGAN Horbaach WHEAT FREE HIGHSTRENGTH 000 PSY...
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,71gSRbyXmoL.jpg,TEARRIFIC LEBENSMITTELECHT Cup HDAY GEPRAGTES ...
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,617Tl40LOXL.jpg,Horbaach HIGHSTRENGTH PSYLLIOM HUSK 1400 PLANT...
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,61BZ4zrjZXL.jpg,COMPOSITION Serving Size:1 Tablet 0.709 g)Each...
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,612mrlqiI4L.jpg,ZRDA' : 11180 ZRDA Qty./Serving EEEE Qty./Serv...
